In [1]:
#clear all objects to save memory
%reset

In [2]:
import scanpy as sc
import numpy as np
# import seaborn as sns
# from matplotlib import pyplot as plt
import anndata2ri
import logging
from scipy.sparse import issparse
from tqdm import tqdm
import os

In [3]:
import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

sc.settings.verbosity = 0
sc.settings.set_figure_params(
    dpi=80,
    facecolor="white",
    # color_map="YlGnBu",
    frameon=False,
)

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

In [4]:
with open('batch_list.txt') as f:
    batch_list = f.readlines()
    
batches = []
for i, b in enumerate(batch_list):
    b = b.strip('/\n')
    # print(f"Processing line {i}: {b}")
    batches.append(b)


In [5]:
lib = '/home/bic/chloes/R/x86_64-pc-linux-gnu-library/4.3'
%R -i lib print(packageVersion("Seurat"))
%R -i lib print(packageVersion("SeuratDisk"))
%R -i lib print(packageVersion("sctransform"))

[1] ‘5.0.1’
[1] ‘0.0.0.9021’
[1] ‘0.4.1’


'value'

In [6]:
%R -i lib print(packageVersion("future"))

[1] ‘1.33.0’


'value'

In [7]:
%R -i lib library(future)

'value'

In [8]:
print(os.environ.get("OPENBLAS_NUM_THREADS"))

None


In [9]:
print("OPENBLAS_NUM_THREADS =", os.environ.get("OPENBLAS_NUM_THREADS"))

OPENBLAS_NUM_THREADS = None


In [ ]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "40" # enable parallelism

In [11]:
%R -i lib library(Seurat)
%R -i lib library(SeuratDisk)
%R -i lib library(sctransform)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

'value'

In [12]:
import pandas as pd

In [13]:
# Load cell annotations
annot = pd.read_csv('cell-annotation.csv')
# Load cell type categories
cells_cat = list(annot['cell.type'].unique())

def overlap(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [14]:
# Load all genes across cell types and get unique set
unique_genes = set()
for cell_type in tqdm(cells_cat, desc='Loading genes'):
    genes_list = np.load(f'./ROSMAP_seurat_QC/by_cell_type/{cell_type}_genes_names.npy', allow_pickle=True)
    unique_genes.update(genes_list)  # add all genes at once
print(f'{len(unique_genes)} unique genes across celltypes')

# Separate MT genes and nuclear genes
MT_genes = sorted([gene for gene in unique_genes if gene.startswith('MT-')])
nuc_genes = sorted(unique_genes - set(MT_genes))  # set difference for nuclear genes

# Find overlap of genes across all cell types
overlap_genes = unique_genes.copy()
for cell_type in tqdm(cells_cat, desc='Computing overlap'):
    genes_list = set(np.load(f'./ROSMAP_seurat_QC/by_cell_type/{cell_type}_genes_names.npy', allow_pickle=True))
    print(f'{cell_type}: {len(genes_list)} genes')
    overlap_genes &= genes_list  # set intersection
    print(f'Current overlap: {len(overlap_genes)}')
overlap_genes = sorted(overlap_genes)

Loading genes: 100%|██████████| 16/16 [00:00<00:00, 552.96it/s]


8079 unique genes across celltypes


Computing overlap: 100%|██████████| 16/16 [00:00<00:00, 544.41it/s]

Endothelial: 7390 genes
Current overlap: 7390
Pericytes: 7337 genes
Current overlap: 7337
SMC: 7379 genes
Current overlap: 7326
Fibroblast: 7337 genes
Current overlap: 7326
Erythrocytes: 7246 genes
Current overlap: 7235
CD8+ T Cells: 6165 genes
Current overlap: 6153
Neutrophils: 6144 genes
Current overlap: 5651
NK Cells: 6578 genes
Current overlap: 5441
Macrophages: 7337 genes
Current overlap: 5441
Microglia: 8079 genes
Current overlap: 5441
Monocytes: 7302 genes
Current overlap: 5440
Astrocyte: 8079 genes
Current overlap: 5440
Inhibitory Neurons: 8079 genes
Current overlap: 5440
Excitatory Neurons: 8079 genes
Current overlap: 5440
OPCs: 8079 genes
Current overlap: 5440
Oligodendrocytes: 8079 genes
Current overlap: 5440


In [15]:
len(overlap_genes)

5440

In [ ]:
for batch in tqdm(batches):
    # Construct the absolute path to the 10x data directory for this batch
    path = os.path.abspath(f"/home/bic/chloes/pool7_z2/ROSMAP_10x_organized/{batch}")
    
    # Load 10x Genomics data into R (via rpy2 magic) as a Seurat-compatible object
    %R -i path rosmap_data <- Read10X(data.dir = path)
    %R seurat_object <- CreateSeuratObject(counts = rosmap_data)
    
    # ---------------- Pre-processing pipeline from Green et al., 2023 ----------------
    # Store the percentage of mitochondrial reads in the Seurat object metadata
    %R seurat_object <- PercentageFeatureSet(seurat_object, pattern = "^MT-", col.name = "percent.mt")
    
    # Remove genes with 0 counts across all cells
    %R options(future.globals.maxSize = 80 * 1024^3)  # Allow up to 80 GB for future operations
    %R plan("multicore", workers = 1)  # Set parallelization to a single worker (multicore)
    %R seurat_object = seurat_object[, unname(which(colSums(GetAssayData(seurat_object)) != 0))]
    
    # Perform SCTransform while regressing out mitochondrial percentage
    # Only use overlapping genes as residual features
    %R -i overlap_genes seurat_object <- SCTransform(seurat_object, residual.features = overlap_genes, vars.to.regress = "percent.mt", verbose = FALSE, conserve.memory = FALSE)

    # Save SCT normalized data 
    %R seurat_object <- DietSeurat(object = seurat_object, counts = T, data = T, scale.data = F, assays = "SCT")
    
    # Define path 
    new_path = f'ROSMAP_seurat_QC_5440/{batch}_seurat_qc.h5Seurat'
    
    # Convert Seurat object to h5ad for Python compatibility
    %R -i new_path as.h5Seurat(seurat_object, new_path, overwrite = TRUE, verbose = TRUE)
    %R -i new_path SaveH5Seurat(seurat_object, filename = new_path, overwrite = TRUE)
    %R -i new_path Convert(new_path, dest = "h5ad", overwrite = TRUE)
    
    # Clean up R environment to free memory before the next iteration
    %R rm(list = ls())
